In [1]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory, TerminationCondition
import psycopg2
import pandas as pd

## Conectar Base de datos y Preparacion de los datos a utilizar

In [2]:
conexion = psycopg2.connect(
        dbname="buap",
        user="postgres",
        password="contrasena",
        host="localhost",
        port="5432"
    )
cursor = conexion.cursor()
datos = {}

In [3]:
# Profesores (con nombre)
cursor.execute("SELECT id_profesor, nombre FROM Profesor LIMIT 50")
profesores = cursor.fetchall()
datos['Id_Profesores'] = [row[0] for row in profesores] # Lista de IDs de profesores
datos['nombres_profesor'] = {row[0]: row[1] for row in profesores} # Diccionario de nombres de profesores


In [4]:
# Materias (con nombre)
cursor.execute("SELECT id_materia, nombre, horas_por_semana, id_tipo_clase FROM Materia LIMIT 50")
materias = cursor.fetchall()
datos['id_Materias'] = [row[0] for row in materias] # Lista de IDs de materias
datos['nombres_materia'] = {row[0]: row[1] for row in materias} # Diccionario de nombres de materias
datos['Horas_Semana_Materia'] = {row[0]: row[2] for row in materias} # Diccionario de horas por semana de cada materia
datos['Tipo_Clase'] = {row[0]: row[3] for row in materias} # Diccionario de tipo de clase de cada materia

In [5]:
# Aulas
cursor.execute("SELECT id_aula, id_edificio, id_tipo_clase FROM Aula LIMIT 50")
rows = cursor.fetchall()

datos['Aulas'] = [(row[0], row[1]) for row in rows] # Lista de IDs compuestos (id_aula, id_edificio)
datos['Tipo_Aula'] = {(row[0], row[1]): row[2] for row in rows}# Diccionario: clave compuesta (id_aula, id_edificio) -> tipo clase


# Bloques horarios
datos['Dia'] = list(range(1, 6)) # Lunes a Viernes
datos['Hora'] = list(range(1, 11)) # 10 bloques horarios por día/ de 8 am a 6 pm

In [6]:
cursor.execute("""
    SELECT A.id_aula, A.id_edificio, M.id_materia,
           CASE WHEN A.id_tipo_clase = M.id_tipo_clase THEN 1 ELSE 0 END AS compatible,
           M.id_tipo_clase AS tipo_materia,
           A.id_tipo_clase AS tipo_aula
    FROM Aula A
    CROSS JOIN Materia M
    WHERE A.id_aula = ANY(%s) AND M.id_materia = ANY(%s)
""", ([a[0] for a in datos['Aulas']], datos['id_Materias']))  # solo ids de aula

datos['Compatibiliadad_Aula_Materia'] = {
    ((row[0], row[1]), row[2]): {
        'compatible': row[3],
        'tipo_materia': row[4],
        'tipo_aula': row[5]
    }
    for row in cursor.fetchall()
}


In [7]:
# Relacionar profesores con materias y viceversa

from collections import defaultdict

# Crear diccionarios como defaultdict
datos['Materias_por_Profesor'] = defaultdict(list)
datos['Profesores_por_Materia'] = defaultdict(list)

# Obtener relaciones profesor-materia desde la tabla 'profesor_materia'
cursor.execute("""
    SELECT id_profesor, id_materia
    FROM profesor_materia
    WHERE id_profesor = ANY(%s) AND id_materia = ANY(%s)
""", (datos['Id_Profesores'], datos['id_Materias']))

# Relacionar profesores con materias y viceversa
asignaciones = cursor.fetchall()
for profe_id, materia_id in asignaciones:
    datos['Materias_por_Profesor'][profe_id].append(materia_id)
    datos['Profesores_por_Materia'][materia_id].append(profe_id)

def asignar_profesores_imaginarios(datos, profesor_imaginario_id):
    """
    Asigna un profesor imaginario a las materias que no tienen profesores asignados.
    
    :param datos: Diccionario que contiene las relaciones entre profesores y materias.
    :param profesor_imaginario_id: ID del profesor imaginario que se asignará.
    """
    # Recorre las materias que no tienen profesores asignados
    for materia_id in datos['id_Materias']:
        if materia_id not in datos['Profesores_por_Materia']:
            # Asigna el profesor imaginario a la materia
            datos['Profesores_por_Materia'][materia_id] = [profesor_imaginario_id]
            datos['Materias_por_Profesor'][profesor_imaginario_id].append(materia_id)

# Definir un ID para el profesor imaginario
profesor_imaginario_id = 'PA'  # Suponiendo que un ID negativo indica un profesor imaginario

# Llamar a la función para asignar profesores imaginarios
asignar_profesores_imaginarios(datos, profesor_imaginario_id)

In [8]:
profesor_imaginario_id = 'PA'

# Si aún no se ha agregado, agrégalo a la lista
if profesor_imaginario_id not in datos['Id_Profesores']:
    datos['Id_Profesores'].append(profesor_imaginario_id)
    datos['nombres_profesor'][profesor_imaginario_id] = "Profesor Imaginario"


In [9]:
print(type(datos))  # Verifica el tipo de 'datos'
print(datos.keys())  # Muestra las claves del diccionario
print(len(datos))    # Muestra cuántos elementos hay en el diccionario


<class 'dict'>
dict_keys(['Id_Profesores', 'nombres_profesor', 'id_Materias', 'nombres_materia', 'Horas_Semana_Materia', 'Tipo_Clase', 'Aulas', 'Tipo_Aula', 'Dia', 'Hora', 'Compatibiliadad_Aula_Materia', 'Materias_por_Profesor', 'Profesores_por_Materia'])
13


## Crear Modelo de Pyomo

In [10]:
# Definir los conjuntos
model = pyo.ConcreteModel()

# Conjuntos
model.Profesores = pyo.Set(initialize=datos['Id_Profesores'])
model.Materias = pyo.Set(initialize=datos['id_Materias'])
model.Aulas = pyo.Set(initialize=[(aula_id, edificio_id) for aula_id, edificio_id in datos['Aulas']])
model.BloquesHorarios = pyo.Set(initialize=[(dia, hora) for dia in datos['Dia'] for hora in datos['Hora']])
model.Dias = pyo.Set(initialize=datos['Dia'])


# Parámetros
model.Horas_Semana_Materia = pyo.Param(model.Materias, initialize=datos['Horas_Semana_Materia'])
model.Tipo_Clase = pyo.Param(model.Materias, initialize=datos['Tipo_Clase'])
model.Tipo_Aula = pyo.Param(model.Aulas, initialize=datos['Tipo_Aula'])
model.Compatibilidad_Aula_Materia = pyo.Param(model.Aulas, model.Materias, initialize={
    (aula, materia): datos['Compatibiliadad_Aula_Materia'].get((aula, materia), {}).get('compatible', 0)
    for aula in model.Aulas for materia in model.Materias
})  
model.Relacion_Profesor_Materia = pyo.Param(model.Profesores, model.Materias, initialize={
    (profe_id, materia_id): 1 if materia_id in datos['Materias_por_Profesor'][profe_id] else 0
    for profe_id in model.Profesores for materia_id in model.Materias
})



In [11]:
# Variables

# Crear índice de combinaciones válidas para X
X_index = [
    (p, m, a, bh)
    for p in datos['Id_Profesores']
    for m in datos['id_Materias']
    if m in datos['Materias_por_Profesor'][p]  # Solo si el profesor puede enseñar la materia
    for a in datos['Aulas']
    if datos['Compatibiliadad_Aula_Materia'].get((a, m), {}).get('compatible', 0) == 1  # Aulas compatibles
    for bh in [(d, h) for d in datos['Dia'] for h in datos['Hora']]
]

model.X = pyo.Var(X_index, domain=pyo.Binary)


# Variables generales: profesor puede enseñar materia (relación)
Y_index = [(p, m) for p in datos['Id_Profesores'] for m in datos['Materias_por_Profesor'][p]]
model.Y = pyo.Var(Y_index, domain=pyo.Binary)


# Variables generales: aula asignada a materia
Z_index = [(a, m) for (a, m), val in datos['Compatibiliadad_Aula_Materia'].items() if val['compatible'] == 1]
model.Z = pyo.Var(Z_index, domain=pyo.Binary)



print(f"Variables reducidas de {len(datos['Id_Profesores'])*len(datos['id_Materias'])*len(datos['Aulas'])*len(datos['Dia'])*len(datos['Hora'])} a {len(X_index)}")


Variables reducidas de 6375000 a 72600


In [12]:
if hasattr(model, 'Restriccion_Horas_Semanales'):
    model.del_component(model.Restriccion_Horas_Semanales)

# Asignar exactamente las horas semanales requeridas por cada materia
def horas_semana_materia_rule(model, m):
    return sum(
        model.X[idx]
        for idx in model.X
        if idx[1] == m  # materia
    ) <= datos['Horas_Semana_Materia'][m]


model.Restriccion_Horas_Semanales = pyo.Constraint(datos['id_Materias'], rule=horas_semana_materia_rule)

In [13]:
# Eliminar la restricción si ya existe
if hasattr(model, 'Restriccion_Clases_No_Corridas'):
    model.del_component(model.Restriccion_Clases_No_Corridas)

MAX_HORAS_CONSECUTIVAS = 3  # por ejemplo

def restriccion_clases_no_corridas_rule(model, p, d, i):
    horas = sorted(datos['Hora'])
    bloques = horas[i:i + MAX_HORAS_CONSECUTIVAS + 1]  # ventana de bloques

    exprs = [
        model.X[idx]
        for idx in model.X
        if idx[0] == p and idx[4] == d and idx[5] in bloques
    ]

    if not exprs:
        return pyo.Constraint.Skip  # No hay clases en esa ventana

    return sum(exprs) <= MAX_HORAS_CONSECUTIVAS




# Índices para las ventanas de tiempo
model.Ventanas = pyo.RangeSet(0, len(datos['Hora']) - MAX_HORAS_CONSECUTIVAS - 1)

model.Restriccion_Clases_No_Corridas = pyo.Constraint(
    model.Profesores, model.Dias, model.Ventanas,
    rule=restriccion_clases_no_corridas_rule
)



# Variable auxiliar que indica si la materia m tiene clase en el día d
model.Materia_Dia_Activa = pyo.Var(model.Materias, model.Dias, domain=pyo.Binary)

# Vincular Materia_Dia_Activa con X: si hay al menos una clase de la materia en el día d, debe ser 1
def materia_dia_activa_rule(model, m, d):
    # Suma de todas las X asignadas a la materia m en día d
    return model.Materia_Dia_Activa[m, d] >= (1.0 / 1000) * sum(
        model.X[idx]
        for idx in model.X
        if idx[1] == m and idx[4] == d
    )
model.Materia_Dia_Activa_Constraint = pyo.Constraint(model.Materias, model.Dias, rule=materia_dia_activa_rule)

# Forzar mínimo días con clases por materia
MIN_DIAS_POR_MATERIA = 5
def min_dias_por_materia_rule(model, m):
    return sum(model.Materia_Dia_Activa[m, d] for d in model.Dias) >= MIN_DIAS_POR_MATERIA

model.Min_Dias_Por_Materia_Constraint = pyo.Constraint(model.Materias, rule=min_dias_por_materia_rule)


In [14]:
from collections import defaultdict

# Agrupar las variables X válidas por materia y día-hora usando X_index
bloques_por_materia_dia = defaultdict(lambda: defaultdict(list))

for (p, m, a, (d, h)) in X_index:
    if datos['Horas_Semana_Materia'][m] > 3:
        bloques_por_materia_dia[m][d].append(((p, m, a, (d, h)), h))  # guardamos también la hora

# Crear las restricciones de ventanas deslizantes
model.restricciones_horas_consecutivas = pyo.ConstraintList()

for m in bloques_por_materia_dia:
    for d in bloques_por_materia_dia[m]:
        # Agrupar todas las variables válidas por hora
        hora_dict = defaultdict(list)
        for idx, h in bloques_por_materia_dia[m][d]:
            hora_dict[h].append(model.X[idx])

        # Para cada ventana de 3 bloques consecutivos (1–3, 2–4, ..., 8–10)
        for h_inicio in range(1, 9):  # de 1 a 8
            ventana_vars = []
            for offset in range(3):
                h_actual = h_inicio + offset
                ventana_vars.extend(hora_dict.get(h_actual, []))
            
            if ventana_vars:
                model.restricciones_horas_consecutivas.add(sum(ventana_vars) <= 2)


In [15]:
from collections import defaultdict

X_por_profesor = defaultdict(list)
for idx in X_index:
    p = idx[0]
    X_por_profesor[p].append(idx)

datos['X_por_profesor'] = X_por_profesor

def max_horas_profesor_rule(model, p):
    if p not in X_por_profesor:
        return pyo.Constraint.Skip
    return sum(model.X[idx] for idx in X_por_profesor[p]) <= 20

model.Restriccion_Max_Horas_Profesor = pyo.Constraint(
    model.Profesores, rule=max_horas_profesor_rule
)


In [16]:
if hasattr(model, 'Restriccion_Profesor_Unico'):
    model.del_component(model.Restriccion_Profesor_Unico)

def no_solapamiento_profesor_rule(model, p, d, h):
    exprs = [
        model.X[idx]
        for idx in model.X
        if idx[0] == p and idx[4] == d and idx[5] == h
    ]
    
    if not exprs:  # No hay clases asignables en este bloque
        return pyo.Constraint.Skip
    
    return sum(exprs) <= 1



model.Restriccion_Profesor_Unico = pyo.Constraint(
    model.Profesores, model.Dias, datos['Hora'],
    rule=no_solapamiento_profesor_rule
)


In [17]:
if hasattr(model, 'Restriccion_Aula_Unica'):
    model.del_component(model.Restriccion_Aula_Unica)


# No asignar un aula a más de una materia en un bloque horario
def no_solapamiento_aula_rule(model, aula_id, edificio_id, dia, hora):
    aula_edificio = (aula_id, edificio_id)
    bloque_horario = (dia, hora)
    
    vars_to_sum = [
        model.X[p, m, aula_edificio, bloque_horario]
        for p in model.Profesores
        for m in model.Materias
        if (p, m, aula_edificio, bloque_horario) in model.X
    ]
    
    if not vars_to_sum:
        return pyo.Constraint.Skip  
    
    return sum(vars_to_sum) <= 1


model.Restriccion_Aula_Unica = pyo.Constraint(
    model.Aulas, model.BloquesHorarios,
    rule=no_solapamiento_aula_rule
)



In [18]:
if hasattr(model, 'Restriccion_Profesor_Compatible'):
    model.del_component(model.Restriccion_Profesor_Compatible)

# Asignar un profesor a una materia solo si el profesor puede enseñar esa materia
def profesor_puede_ensenar_rule(model, p, m, id_aula, id_edificio, dia, hora):
    aula = (id_aula, id_edificio)
    bloque_horario = (dia, hora)
    return model.X[p, m, aula, bloque_horario] <= model.Relacion_Profesor_Materia[p, m]


model.Restriccion_Profesor_Compatible = pyo.Constraint(
    X_index,
    rule=profesor_puede_ensenar_rule
)



In [19]:
if hasattr(model, 'Restriccion_Aula_Compatible'):
    model.del_component(model.Restriccion_Aula_Compatible)

# Asignar un aula a una materia solo si el tipo de aula es compatible con el tipo de materia
def aula_compatible_materia_rule(model, p, m, aula_id, edificio_id, dia, hora):
    aula = (aula_id, edificio_id)
    bloque_horario = (dia, hora)
    return model.X[p, m, aula, bloque_horario] <= model.Compatibilidad_Aula_Materia[aula, m]


model.Restriccion_Aula_Compatible = pyo.Constraint(
    X_index,
    rule=aula_compatible_materia_rule
)


In [20]:
# %% Restricciones que vinculan variables combinadas con variables generales

if hasattr(model, 'Relacion_X_Y'):
    model.del_component(model.Relacion_X_Y)

def relacion_X_Y_rule(model, p, m, aula_id, edificio_id, dia, hora):
    aula = (aula_id, edificio_id)
    bloque_horario = (dia, hora)
    return model.X[p, m, aula, bloque_horario] <= model.Y[p, m]

model.Relacion_X_Y = pyo.Constraint(
    X_index,
    rule=relacion_X_Y_rule
)

if hasattr(model, 'Relacion_X_Z'):
    model.del_component(model.Relacion_X_Z)

def relacion_X_Z_rule(model, p, m, aula_id, edificio_id, dia, hora):
    aula = (aula_id, edificio_id)
    bloque_horario = (dia, hora)
    return model.X[p, m, aula, bloque_horario] <= model.Z[aula, m]

model.Relacion_X_Z = pyo.Constraint(
    X_index,
    rule=relacion_X_Z_rule
)


In [21]:
# Restricciones adicionales para asegurar que cada materia tenga asignado al menos un profesor y un aula

if hasattr(model, 'Materia_Tiene_Profesor'):
    model.del_component(model.Materia_Tiene_Profesor)

# Crear diccionario que mapea cada materia a la lista de profesores que la pueden enseñar
profesores_por_materia = {m: [] for m in model.Materias}
for (p, m) in model.Y.index_set():
    profesores_por_materia[m].append(p)

# Ahora la restricción
def materia_tiene_profesor_rule(model, m):
    return sum(model.Y[p, m] for p in profesores_por_materia[m]) >= 1

model.Materia_Tiene_Profesor = pyo.Constraint(model.Materias, rule=materia_tiene_profesor_rule)

In [22]:
def materia_tiene_aula_rule(model, m):
    # Filtra solo índices con materia m
    relevant_indices = [idx for idx in model.Z.index_set() if idx[-1] == m]
    return sum(model.Z[idx] for idx in relevant_indices) >= 1


if hasattr(model, 'Materia_Tiene_Aula'):
    model.del_component(model.Materia_Tiene_Aula)

model.Materia_Tiene_Aula = pyo.Constraint(model.Materias, rule=materia_tiene_aula_rule)


In [23]:
# Objetivo compuesto: minimizar el número de profesores, aulas y maximizar el uso horario usando una funcion objetivo ponderada

# Parámetros de ponderación
peso_profesores = 1.0
peso_aulas = 1.0
peso_asignaciones = -0.1
peso_distribucion = -0.05  # Penalización negativa para maximizar distribución

def nuevo_objetivo_compuesto_rule(model):
    total_profesores = sum(model.Y[idx] for idx in model.Y.index_set())
    total_aulas = sum(model.Z[idx] for idx in model.Z.index_set())
    total_asignaciones = sum(model.X[idx] for idx in model.X.index_set())
    
    # Calcular varianza en la distribución de clases
    distribucion = 0
    for p in model.Profesores:
        horas_por_dia = {d: 0 for d in model.Dias}
        for idx in model.X:
            if idx[0] == p and model.X[idx].value is not None:  # Considerar solo variables definidas
                d = idx[3][0]  # Día de la asignación
                horas_por_dia[d] += 1
        # Premiar distribución uniforme (mínima varianza)
        media = sum(horas_por_dia.values()) / len(horas_por_dia)
        varianza = sum((h - media)**2 for h in horas_por_dia.values()) / len(horas_por_dia)
        distribucion += varianza
        
    return (
        peso_profesores * total_profesores +
        peso_aulas * total_aulas +
        peso_asignaciones * total_asignaciones +
        peso_distribucion * distribucion
    )

model.Objetivo = pyo.Objective(rule=nuevo_objetivo_compuesto_rule, sense=pyo.minimize)


In [24]:

def resolver_modelo(model, mipgap=0.02, threads=4, emphasis=1):
    from pyomo.opt import SolverFactory, TerminationCondition

    solver = SolverFactory('cplex')

    if not solver.available():
        raise RuntimeError("CPLEX no está disponible. Verifica la instalación o el PATH.")

    resultado = solver.solve(model, tee=True)

    # Verificar si la solución fue óptima o factible
    condicion = resultado.solver.termination_condition
    if condicion in [TerminationCondition.optimal, TerminationCondition.feasible]:
        print(f"Solución encontrada: {condicion}")
    else:
        print(f"Problema durante la resolución: {condicion}")
        print("Revisa el modelo o ajusta parámetros del solver.")

    return resultado


In [25]:
def mostrar_bloque(dia, hora):
    dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']
    dia_str = dias[dia - 1]  # porque 1 = Lunes
    hora_inicio = 7 + hora  # si hora=1 corresponde a 8am
    hora_fin = hora_inicio + 1
    return f"{dia_str} {hora_inicio:02d}:00 - {hora_fin:02d}:00"

def mostrar_asignacion(model):
    print("\nAsignaciones de profesores, materias, aulas y horarios:")
    for idx in model.X:
        if pyo.value(model.X[idx]) > 0.5:
            # idx = (p, m, aula, edificio, dia, hora)
            p, m, aula, edificio, dia, hora = idx
            print(f"Profesor: {datos['nombres_profesor'].get(p, p)} | "
                  f"Materia: {datos['nombres_materia'].get(m, m)} | "
                  f"Aula: {aula} (Edificio {edificio}) | "
                  f"Horario: {mostrar_bloque(dia, hora)}")



In [26]:
# Resolver el modelo
resultados = resolver_modelo(model)

# Verificar si encontró solución
if (resultados.solver.status == pyo.SolverStatus.ok and
    resultados.solver.termination_condition == TerminationCondition.optimal):

    print("Solución óptima encontrada.\n")
    mostrar_asignacion(model)

else:
    print("No se encontró solución óptima.")
    print("Estado:", resultados.solver.status)
    print("Condición de terminación:", resultados.solver.termination_condition)



Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\bdgae\AppData\Local\Temp\tmpdo71iys_.cplex.log' open.
CPLEX> Problem 'C:\Users\bdgae\AppData\Local\Temp\tmp_p38g8rx.pyomo.lp' read.
Read time = 0.44 sec. (33.85 ticks)
CPLEX> Problem name         : C:\Users\bdgae\AppData\Local\Temp\tmp_p38g8rx.pyomo.lp
Objective sense      : Minimize
Variables            :   74191  [Binary: 74191]
Objective nonzeros   :   73941
Linear constraints   :  294348  [Less: 294198,  Greater: 150]
  Nonzeros           : 1053161
  RHS nonzeros       :  148898

Variables            : Min LB: 0.000000         Max UB: 1.000000       
Objective nonzeros   : Min   :